# Суммаризация новостей
Входные данные: новости (таблица df)
1. Создайте новый столбец в таблице – summary и поместите в него первое предложение из текста.
2. Сгенерируйте саммари каждого текста, используя gensim.summarize()
3. Используя nltk.translate.bleu_score.sentence_bleu([true_summary], generated_summary), оцените качество сгенерированных саммари 
4. Вычислите средний BLEU по всей коллекции текстов
5. Лемматизация и удаление стоп-слов – на ваше усмотрение

In [28]:
import pandas as pd

df = pd.read_csv('news.csv')
df.head()

,text,date,event,class
0,В ПЕТЕРБУРГЕ ПРОШЕЛ МИТИНГ ПРОТИВ ПЕРЕДАЧИ ИС...,2017-01-10,Передача РПЦ Исаакиевского собора,Внутренняя политика РФ
1,"Lenta.co, Москва, 14 января 2017 СИТУАЦИЯ С П...",2017-01-10,Передача РПЦ Исаакиевского собора,Внутренняя политика РФ
2,"Аргументы и Факты (aif.ru), Москва, 14 января...",2017-01-10,Передача РПЦ Исаакиевского собора,Внутренняя политика РФ
3,"Google Новости ТОП, Москва, 14 января 2017 АК...",2017-01-10,Передача РПЦ Исаакиевского собора,Внутренняя политика РФ
4,"Газета.Ru, Москва, 13 января 2017 В МОСКОВСКО...",2017-01-10,Передача РПЦ Исаакиевского собора,Внутренняя политика РФ


In [18]:
import re
regex = re.compile("[А-Яа-я]+")
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


def split_sents(text):
    return sent_tokenize(text)


def  remove_stopwords(text, mystopwords = stopwords.words('russian')):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return " "
    
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [31]:
from nltk.tokenize import sent_tokenize
summaries = []
for index, row in df.iterrows():

    text = row.text
    sents = sent_tokenize(text)
    if '2017' in sents[0]:
        summary = sents[0].split('2017')[1]
    else:
        summary = sents[0]
    summaries.append(summary) 
df['summary'] = summaries

In [36]:
df.text = df.text.str.lower()
df.text = df.text.apply(remove_stopwords) 
df.text = df.text.apply(lemmatize)

In [37]:
df.summary = df.summary.str.lower()
df.summary = df.summary.apply(remove_stopwords) 
df.summary = df.summary.apply(lemmatize)

In [45]:
from gensim.summarization import summarize
import nltk

text = df.iloc[100].text
generated_summary = summarize(text, ratio = 0.05)

BLEUscore = nltk.translate.bleu_score.sentence_bleu([df.iloc[100].summary], generated_summary)



год здание капитолий вашингтон готовый инаугурация новый президент сша.

инаугурация 45-го президент сша дональд трамп полдень 20 январь (20:00 мск) миллиардер трамп становиться хозяин белый дом, сменить пост барак обама ранний утро 20 январь 2017 год здание капитолий вашингтон готовый инаугурация новый президент сша.
0.28421976445468305
